In [4]:
import keras
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras import regularizers
from tensorflow.keras.regularizers import l2
import os
import pickle
import time
import matplotlib.pyplot as plt
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

#os.environ["OMP_NUM_THREADS"] = '6'
#export TF_ENABLE_ONEDNN_OPTS=1
#export TF_ENABLE_MKL_NATIVE_FORMAT=1
#tf.config.threading.set_inter_op_parallelism_threads(3) 
#tf.config.threading.set_intra_op_parallelism_threads(3)
#tf.config.set_soft_device_placement(enabled = True)
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' #to hide tensorflow warnings

Num GPUs Available:  0


In [2]:
#input data function
def load_input_data(input_file):
    with open(input_file, 'rb') as file:
        dataset = pickle.load(file)
        # For our problem we have positions and energy arrays,
        # which are respectively data and labels in common Machine Learning language
        positions_array = np.array([mol['positions'] for mol in dataset])
        energies_array = np.array([mol['energy'] for mol in dataset])

    return positions_array, energies_array

#model creation
def createmodel(input_class, output_class, base_neurons):
    model = Sequential([
        Dense(input_class)
        Dense(base_neurons, activation='relu', kernel_regularizer=l2(0.01)),
        Dense(base_neurons/2., activation='relu', kernel_regularizer=l2(0.01)),
        Dense(base_neurons/4., activation='gelu', kernel_regularizer=l2(0.01)),
        Dense(base_neurons/8., activation='gelu', kernel_regularizer=l2(0.01)),
        Dense(base_neurons/16., activation='relu', kernel_regularizer=l2(0.01)),
        Dense(output_class)
    ])
    return model

#model compilation
def compilemodel(model):
    #optimizer
    opt = keras.optimizers.Adam() #learning_rate=LR
    #optimizer training with type of dataset
    model.compile(
        #rms
        loss='mean_squared_error',
        #adam with LR and decay
        optimizer=opt,
        #metrics to monitor
        metrics=['mean_squared_error', 'mae']
    )

In [3]:
#define input train and test data directory
#input_file = 'saved_dataset/merged_dataset_rand_5a.pickle'
input_file = 'saved_dataset/merged_randcartp06_dataset_5a.pickle'
#Load dataset into data and labels (positions=data, energy=labels)
positions_array, energies_array = load_input_data(input_file)

# Splitting the dataset into training, validation, and test sets
# with a 72-20-8% ratio from initial dataset
train_val_data, test_data, train_val_labels, test_labels = train_test_split(
    positions_array, energies_array, test_size=0.01, random_state=42)
# Further splitting train, into new train and validation samples
# Takes 80% and is divided to 10% of that for the validation set
train_data, val_data, train_labels, val_labels = train_test_split(
    train_val_data, train_val_labels, test_size=0.1, random_state=42)

print(train_data.shape[0], 'train samples')
print(test_data.shape[0], 'test samples')
print(val_data.shape[0], 'validation samples')
print(train_data.shape[1], 'train shape')
print(train_data)

9714 train samples
110 test samples
1080 validation samples
3 train shape
[[0.95597293 2.59722054 1.97978283]
 [3.34171704 1.41442163 2.10516884]
 [1.11177492 1.39656736 1.08879067]
 ...
 [0.97968797 1.5170054  2.96534365]
 [3.28015319 2.36115725 1.49903299]
 [2.48877853 1.15783563 1.14141772]]


In [4]:
#define input, output dimensions
input_class = train_data.shape[1] #dH1,dH2,theta, in Angstrom units
output_class = 1 #total system energy in eV
n_iterations = 1
#define learing rate, epochs and batch_size for optimizer and fitter
batch_size = 16
epochs = 1000
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=50)
history = []
neurons = 256
end_neurons = 512

In [5]:
for i in range(0,n_iterations):
    # Base model creation
    #iterations = n_iterations
    #increment = (end_neurons- base_neurons) // (iterations-1)
    #neurons = base_neurons + i*increment
    model = createmodel(input_class, output_class, neurons)
    #neurons = neurons*2
    
    #model name for saving purposes
    ainame = 'conocart'
    n_layers = len(model.layers)
    model_name = 'topbs16_'+str(ainame)+'_'+str(n_layers)+'layers_'+str(neurons)+'neurons_'+''
    
    # Model compilation with automatic LR by adam
    compilemodel(model)
    # Model training 
    fitstart = time.time()
    history.append( 
        model.fit(
        train_data, train_labels, epochs=epochs, batch_size=batch_size, 
        validation_data=(val_data, val_labels), 
        callbacks=callback, verbose = 1
             )
                  )
    fitend = time.time()
    print(f'Time passed for fit {i+1}: {fitend-fitstart:.1f} seconds')
    
    # Save model and weights
    #create directory to save weighted model
    save_dir = os.path.join(os.getcwd(), 'saved_model')
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    model_path = os.path.join(save_dir, model_name + '.keras')
    model.save(model_path)
    print('Saved trained model at %s ' % model_path , '\n')
    
    #duration = 3  # seconds
    #freq = 440  # Hz
    #os.system('play -nq -t alsa synth {} sine {}'.format(duration, freq))
    #print('\007')

Epoch 1/2500


2024-02-28 20:28:41.078314: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


608/608 [==============================] - 2s 1ms/step - loss: 20772.7383 - mean_squared_error: 20769.5781 - mae: 83.8821 - val_loss: 103.1401 - val_mean_squared_error: 98.9575 - val_mae: 8.2670
Epoch 2/2500
608/608 [==============================] - 1s 1ms/step - loss: 20.1019 - mean_squared_error: 15.8592 - mae: 2.5905 - val_loss: 10.7725 - val_mean_squared_error: 6.5604 - val_mae: 2.2110
Epoch 3/2500
608/608 [==============================] - 1s 1ms/step - loss: 7.7445 - mean_squared_error: 3.5583 - mae: 1.3298 - val_loss: 11.3094 - val_mean_squared_error: 7.1456 - val_mae: 2.2377
Epoch 4/2500
608/608 [==============================] - 1s 1ms/step - loss: 7.0125 - mean_squared_error: 2.8816 - mae: 1.2310 - val_loss: 6.1048 - val_mean_squared_error: 2.0103 - val_mae: 0.9077
Epoch 5/2500
608/608 [==============================] - 1s 1ms/step - loss: 6.4913 - mean_squared_error: 2.4308 - mae: 1.1331 - val_loss: 5.8593 - val_mean_squared_error: 1.8307 - val_mae: 1.0175
Epoch 6/2500
608/

Epoch 84/2500
608/608 [==============================] - 1s 1ms/step - loss: 2.5624 - mean_squared_error: 1.1671 - mae: 0.7902 - val_loss: 2.6487 - val_mean_squared_error: 1.2555 - val_mae: 1.0843
Epoch 85/2500
608/608 [==============================] - 1s 1ms/step - loss: 3.0517 - mean_squared_error: 1.6625 - mae: 0.8260 - val_loss: 1.8272 - val_mean_squared_error: 0.4412 - val_mae: 0.5178
Epoch 86/2500
608/608 [==============================] - 1s 1ms/step - loss: 2.1915 - mean_squared_error: 0.8114 - mae: 0.6750 - val_loss: 1.5865 - val_mean_squared_error: 0.2096 - val_mae: 0.3611
Epoch 87/2500
608/608 [==============================] - 1s 1ms/step - loss: 2.3265 - mean_squared_error: 0.9545 - mae: 0.7293 - val_loss: 3.4572 - val_mean_squared_error: 2.0905 - val_mae: 1.4153
Epoch 88/2500
608/608 [==============================] - 1s 1ms/step - loss: 2.4081 - mean_squared_error: 1.0454 - mae: 0.7912 - val_loss: 6.2296 - val_mean_squared_error: 4.8721 - val_mae: 2.1155
Epoch 89/2500
6

608/608 [==============================] - 1s 1ms/step - loss: 1.7076 - mean_squared_error: 0.7755 - mae: 0.6775 - val_loss: 1.1105 - val_mean_squared_error: 0.1795 - val_mae: 0.3154
Epoch 167/2500
608/608 [==============================] - 1s 1ms/step - loss: 1.8054 - mean_squared_error: 0.8770 - mae: 0.6917 - val_loss: 2.4058 - val_mean_squared_error: 1.4785 - val_mae: 1.1859
Epoch 168/2500
608/608 [==============================] - 1s 1ms/step - loss: 1.6667 - mean_squared_error: 0.7407 - mae: 0.6490 - val_loss: 6.1968 - val_mean_squared_error: 5.2717 - val_mae: 2.2754
Epoch 169/2500
608/608 [==============================] - 1s 1ms/step - loss: 1.7756 - mean_squared_error: 0.8518 - mae: 0.6750 - val_loss: 1.0283 - val_mean_squared_error: 0.1069 - val_mae: 0.2590
Epoch 170/2500
608/608 [==============================] - 1s 1ms/step - loss: 1.8508 - mean_squared_error: 0.9324 - mae: 0.7220 - val_loss: 2.7748 - val_mean_squared_error: 1.8561 - val_mae: 1.3205
Epoch 171/2500
608/608 [=

...

608/608 [==============================] - 1s 1ms/step - loss: 0.4267 - mean_squared_error: 0.1532 - mae: 0.3132 - val_loss: 0.7347 - val_mean_squared_error: 0.4613 - val_mae: 0.6558
Epoch 2135/2500
608/608 [==============================] - 1s 1ms/step - loss: 0.4087 - mean_squared_error: 0.1353 - mae: 0.2906 - val_loss: 0.3051 - val_mean_squared_error: 0.0318 - val_mae: 0.1162
Epoch 2136/2500
608/608 [==============================] - 1s 1ms/step - loss: 0.4946 - mean_squared_error: 0.2211 - mae: 0.3500 - val_loss: 0.3347 - val_mean_squared_error: 0.0615 - val_mae: 0.1685
Epoch 2137/2500
608/608 [==============================] - 1s 1ms/step - loss: 0.4259 - mean_squared_error: 0.1531 - mae: 0.3023 - val_loss: 0.3666 - val_mean_squared_error: 0.0936 - val_mae: 0.2487
Epoch 2138/2500
608/608 [==============================] - 1s 1ms/step - loss: 0.4413 - mean_squared_error: 0.1680 - mae: 0.3093 - val_loss: 1.2338 - val_mean_squared_error: 0.9602 - val_mae: 0.9613
Epoch 2139/2500
608/6

608/608 [==============================] - 1s 1ms/step - loss: 0.4663 - mean_squared_error: 0.1957 - mae: 0.3413 - val_loss: 0.9335 - val_mean_squared_error: 0.6625 - val_mae: 0.7888
Epoch 2217/2500
608/608 [==============================] - 1s 1ms/step - loss: 0.4128 - mean_squared_error: 0.1420 - mae: 0.2931 - val_loss: 0.4625 - val_mean_squared_error: 0.1914 - val_mae: 0.4050
Epoch 2218/2500
608/608 [==============================] - 1s 1ms/step - loss: 0.4081 - mean_squared_error: 0.1373 - mae: 0.2900 - val_loss: 0.4197 - val_mean_squared_error: 0.1487 - val_mae: 0.3244
Epoch 2219/2500
608/608 [==============================] - 1s 1ms/step - loss: 0.4177 - mean_squared_error: 0.1468 - mae: 0.2967 - val_loss: 1.1197 - val_mean_squared_error: 0.8490 - val_mae: 0.8955
Epoch 2220/2500
608/608 [==============================] - 1s 1ms/step - loss: 0.4148 - mean_squared_error: 0.1441 - mae: 0.2929 - val_loss: 0.3049 - val_mean_squared_error: 0.0343 - val_mae: 0.1266
Epoch 2221/2500
608/6

608/608 [==============================] - 1s 1ms/step - loss: 0.4278 - mean_squared_error: 0.1627 - mae: 0.3187 - val_loss: 1.1702 - val_mean_squared_error: 0.9055 - val_mae: 0.9236
Epoch 2299/2500
608/608 [==============================] - 1s 1ms/step - loss: 0.4110 - mean_squared_error: 0.1457 - mae: 0.3006 - val_loss: 0.3413 - val_mean_squared_error: 0.0756 - val_mae: 0.2402
Epoch 2300/2500
608/608 [==============================] - 1s 1ms/step - loss: 0.4035 - mean_squared_error: 0.1379 - mae: 0.2911 - val_loss: 0.5585 - val_mean_squared_error: 0.2927 - val_mae: 0.4939
Epoch 2301/2500
608/608 [==============================] - 1s 1ms/step - loss: 0.3764 - mean_squared_error: 0.1112 - mae: 0.2604 - val_loss: 0.4873 - val_mean_squared_error: 0.2222 - val_mae: 0.4324
Epoch 2302/2500
608/608 [==============================] - 1s 1ms/step - loss: 0.4153 - mean_squared_error: 0.1501 - mae: 0.3022 - val_loss: 0.3192 - val_mean_squared_error: 0.0540 - val_mae: 0.1872
Epoch 2303/2500
608/6

608/608 [==============================] - 1s 1ms/step - loss: 0.4237 - mean_squared_error: 0.1618 - mae: 0.3047 - val_loss: 0.3432 - val_mean_squared_error: 0.0814 - val_mae: 0.2241
Epoch 2381/2500
608/608 [==============================] - 1s 1ms/step - loss: 0.4071 - mean_squared_error: 0.1453 - mae: 0.2981 - val_loss: 0.3482 - val_mean_squared_error: 0.0863 - val_mae: 0.2410
Epoch 2382/2500
608/608 [==============================] - 1s 1ms/step - loss: 0.4014 - mean_squared_error: 0.1394 - mae: 0.2904 - val_loss: 0.4250 - val_mean_squared_error: 0.1625 - val_mae: 0.3538
Epoch 2383/2500
608/608 [==============================] - 1s 1ms/step - loss: 0.3777 - mean_squared_error: 0.1157 - mae: 0.2671 - val_loss: 0.3638 - val_mean_squared_error: 0.1017 - val_mae: 0.2843
Epoch 2384/2500
608/608 [==============================] - 1s 1ms/step - loss: 0.4300 - mean_squared_error: 0.1682 - mae: 0.3271 - val_loss: 0.3632 - val_mean_squared_error: 0.1014 - val_mae: 0.2652
Epoch 2385/2500
608/6

608/608 [==============================] - 1s 1ms/step - loss: 0.3742 - mean_squared_error: 0.1158 - mae: 0.2669 - val_loss: 0.4732 - val_mean_squared_error: 0.2150 - val_mae: 0.4316
Epoch 2463/2500
608/608 [==============================] - 1s 1ms/step - loss: 0.4111 - mean_squared_error: 0.1534 - mae: 0.3053 - val_loss: 0.3206 - val_mean_squared_error: 0.0628 - val_mae: 0.1786
Epoch 2464/2500
608/608 [==============================] - 1s 1ms/step - loss: 0.4091 - mean_squared_error: 0.1512 - mae: 0.3042 - val_loss: 0.4271 - val_mean_squared_error: 0.1689 - val_mae: 0.3535
Epoch 2465/2500
608/608 [==============================] - 1s 1ms/step - loss: 0.3671 - mean_squared_error: 0.1091 - mae: 0.2565 - val_loss: 0.3264 - val_mean_squared_error: 0.0683 - val_mae: 0.2039
Epoch 2466/2500
608/608 [==============================] - 1s 1ms/step - loss: 0.3987 - mean_squared_error: 0.1406 - mae: 0.2980 - val_loss: 0.3079 - val_mean_squared_error: 0.0496 - val_mae: 0.1418
Epoch 2467/2500
608/6

In [6]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               1024      
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
 dense_4 (Dense)             (None, 16)                528       
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 44801 (175.00 KB)
Trainable params: 44801 

In [ ]:
#plotting test and predictions for various scenarios

#setting bigger font to ease the eyes
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

    # Estrazione dell'andamento dell'errore sul set di addestramento e sul set di validazione
    train_loss = history[i].history['loss']
    val_loss = history[i].history['val_loss']

    # Creazione del grafico di errore di training e validazione
    plot_dir = os.path.join(os.getcwd(), 'model_plots')
    if not os.path.isdir(plot_dir):
        os.makedirs(plot_dir)
    plot_name = model_name + '__errplot.png'
    plot_path = os.path.join(plot_dir, plot_name)
    
    plt.figure(figsize=(15, 10))
    plt.plot(train_loss, label='Errore sul set di addestramento')
    plt.plot(val_loss, label='Errore sul set di validazione')
    plt.xlabel('Epoche')
    plt.ylabel('Errore')
    plt.ylim(-1, 10)
    plt.grid(axis='both')
    plt.legend()
    plt.title('Andamento dell\'errore')
    plt.savefig(plot_path)

    
    #O_HH plot save
    input_file = 'saved_dataset/dataset_O_HH_100_5a_test.pickle' 

    positions, energies = load_input_data(input_file)
    d1 = positions[:,0] #dH1>dH2
    d2 = positions[:,1]
    theta = positions[:,2]

    input_data = positions
    tf.function(autograph=False, reduce_retracing=True)
    # Effettua previsioni sulle nuove osservazioni
    predictions = model.predict(input_data)

    # Creazione del grafico di O_HH
    graph_dir = os.path.join(os.getcwd(), 'model_graphs_O_HH')
    if not os.path.isdir(graph_dir):
        os.makedirs(graph_dir)
    graph1_name = model_name + '__O_HHfit.png'
    graph1_path = os.path.join(graph_dir, graph1_name)
    
    fig1, ax1 = plt.subplots(figsize=(21.4,12))
    ax1.set(title='Predizione AI per O_HH: 0.1<theta<pi/2 e dH1=dH2= 2A')
    ax1.set(xlabel="Theta [rad]", ylabel="Energy [eV]")
    ax1.scatter(theta, energies, color ='salmon', label='pw.x energies')
    ax1.scatter(theta, predictions, color='darkcyan', label='AI predictions')
    ax1.grid(axis='both')
    ax1.legend(loc='best', ncol=1, facecolor= 'white')
    fig1.savefig(graph1_path, dpi = 200)

    
    #theta plot save
    input_file2 = 'saved_dataset/dataset_theta_100_5a_test.pickle' 

    positions2, energies2 = load_input_data(input_file2)
    d12 = positions2[:,0] #dH1>dH2
    d22 = positions2[:,1]
    theta2 = positions2[:,2]

    input_data2 = positions2
    # Effettua previsioni sulle nuove osservazioni
    predictions2 = model.predict(input_data2)

    # Creazione del grafico di theta
    graph2_dir = os.path.join(os.getcwd(), 'model_graphs_theta')
    if not os.path.isdir(graph2_dir):
        os.makedirs(graph2_dir)
    graph2_name = model_name + '__thetafit.png'
    graph2_path = os.path.join(graph2_dir, graph2_name)
    
    fig2, ax2 = plt.subplots(figsize=(21.4,12))
    ax2.set(title='Predizione AI per 1<theta<pi e dH1=dH2= 0.96A')
    ax2.set(xlabel="Theta [rad]", ylabel="Energy [eV]")
    ax2.scatter(theta2, energies2, color ='salmon', label='pw.x energies')
    ax2.scatter(theta2, predictions2, color='darkcyan', label='AI predictions')
    ax2.grid(axis='both')
    ax2.legend(loc='best', ncol=1, facecolor= 'white')
    fig2.savefig(graph2_path, dpi = 200)

    
    #dh2 plot save
    input_file3 = 'saved_dataset/dataset_dh2_100_5a_test.pickle' 

    positions3, energies3 = load_input_data(input_file3)
    d13 = positions3[:,0] #dH1>dH2
    d23 = positions3[:,1]
    theta3 = positions3[:,2]

    input_data3 = positions3
    # Effettua previsioni sulle nuove osservazioni
    predictions3 = model.predict(input_data3)

    # Creazione del grafico di theta
    graph3_dir = os.path.join(os.getcwd(), 'model_graphs_dh2')
    if not os.path.isdir(graph3_dir):
        os.makedirs(graph3_dir)
    graph3_name = model_name + '__dh2fit.png'
    graph3_path = os.path.join(graph3_dir, graph3_name)
    
    fig3, ax3 = plt.subplots(figsize=(21.4,12))
    ax3.set(title='Predizione AI per 0.6<dH2<3.6, dH1=0.96A, theta=1.82rad')
    ax3.set(xlabel="dH2 [Angstrom]", ylabel="Energy [eV]")
    ax3.scatter(d23, energies3, color ='salmon', label='pw.x energies')
    ax3.scatter(d23, predictions3, color='darkcyan', label='AI predictions')
    ax3.grid(axis='both')
    ax3.legend(loc='best', ncol=1, facecolor= 'white')
    fig3.savefig(graph3_path, dpi = 200)
    
    #OH_H plot save
    input_file4 = 'saved_dataset/dataset_OH_H_100_5a_test.pickle' 

    positions4, energies4 = load_input_data(input_file4)
    d14 = positions4[:,0] #dH1>dH2
    d24 = positions4[:,1]
    theta4 = positions4[:,2]

    input_data4 = positions4
    # Effettua previsioni sulle nuove osservazioni
    predictions4 = model.predict(input_data4)

    # Creazione del grafico di OH_H
    graph4_dir = os.path.join(os.getcwd(), 'model_graphs_OH_H')
    if not os.path.isdir(graph4_dir):
        os.makedirs(graph4_dir)
    graph4_name = model_name + '__OH_Hfit.png'
    graph4_path = os.path.join(graph4_dir, graph4_name)
    
    fig4, ax4 = plt.subplots(figsize=(21.4,12))
    ax4.set(title='Predizione AI per OH_H: 0.6<dH1<2, theta=1.82rad e dH2= 3A')
    ax4.set(xlabel="dH1 [Angstrom]", ylabel="Energy [eV]")
    ax4.scatter(d14, energies4, color ='salmon', label='pw.x energies')
    ax4.scatter(d14, predictions4, color='darkcyan', label='AI predictions')
    ax4.grid(axis='both')
    ax4.legend(loc='best', ncol=1, facecolor= 'white')
    fig4.savefig(graph4_path, dpi = 200)  
    
        #OH_H plot save
    input_file5 = 'saved_dataset/dataset_O_H_H_100_5a_test.pickle' 

    positions5, energies5 = load_input_data(input_file5)
    d15 = positions5[:,0] #dH1>dH2
    d25 = positions5[:,1]
    theta5 = positions5[:,2]

    input_data5 = positions5
    # Effettua previsioni sulle nuove osservazioni
    predictions5 = model.predict(input_data5)

    # Creazione del grafico di OH_H
    graph5_dir = os.path.join(os.getcwd(), 'model_graphs_O_H_H')
    if not os.path.isdir(graph5_dir):
        os.makedirs(graph5_dir)
    graph5_name = model_name + '__O_H_Hfit.png'
    graph5_path = os.path.join(graph5_dir, graph5_name)
    
    fig5, ax5 = plt.subplots(figsize=(21.4,12))
    ax5.set(title='Predizione AI per O_H_H: 0.5<dH1<3.5 A, theta=0.37 rad e dH2= 2A')
    ax5.set(xlabel="dH1 [Angstrom]", ylabel="Energy [eV]")
    ax5.scatter(d15, energies5, color ='salmon', label='pw.x energies')
    ax5.scatter(d15, predictions5, color='darkcyan', label='AI predictions')
    ax5.grid(axis='both')
    ax5.legend(loc='best', ncol=1, facecolor= 'white')
    fig5.savefig(graph5_path, dpi = 200)  
    
    plt.close('all')